### Copy data to current server

In [ ]:
import os

# Remote data directory
remote_data_dir = "/data/2m_temperature_GL_2015-2024.nc"
remote_access = "uribe055@cs-u-spatial-513.cs.umn.edu:" + remote_data_dir

# Local data directory
home_dir = "/home/uribe-55/sedona_experiments"
local_data_dir = os.path.join(home_dir, "unprocessed_data")
local_access = "uribe055@cs-spatial-501.cs.umn.edu:" + local_data_dir

In [ ]:
scp ${remote_access} ${local_access}

### Prepare data

In [ ]:
import xarray as xr
import os

# Get latest 5 years of data
ds_full = xr.open_dataset(os.path.join(local_data_dir, "2m_temperature_GL_2015-2024.nc"))
ds = ds_full.sel(time=slice('2019-01-01', '2024-12-31'))
ds = ds.rename({"valid_time": "time", "2tm": "2m_temperature"})

# No splitting data
ds.to_netcdf(os.path.join(home_dir, f"processed_data/2m_temperature_GL_2015-2024.nc"))

# Split data into daily .nc files
# for t in ds.time[::24]:
#     date = str(t.values)[:10]
#     ds_single = ds.sel(time=t)
#     ds_single.to_netcdf(os.path.join(home_dir, f"processed_data/t2m_{date}.nc"))